<a href="https://colab.research.google.com/github/ccunique/ds_code_examples/blob/main/Graph/GNN/DGL/hetergraph_graph_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Oct 25 08:16:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install  dgl -f https://data.dgl.ai/wheels/cu118/repo.html

Looking in links: https://data.dgl.ai/wheels/cu118/repo.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.3/91.3 MB 8.1 MB/s eta 0:00:00


In [5]:
import torch
import torch as th
import dgl

In [6]:
torch.__version__, dgl.__version__

('2.1.0+cu118', '1.1.2+cu118')

# test batch hetergraph

In [7]:
hg1 = dgl.heterograph({
    ('user', 'plays', 'game') : (th.tensor([0, 1]), th.tensor([0, 0]))})

hg2 = dgl.heterograph({
    ('user', 'plays', 'game') : (th.tensor([0, 0, 0]), th.tensor([1, 0, 2]))})

hg3 = dgl.heterograph({
    ('user', 'follows', 'user') : (th.tensor([0]), th.tensor([1]))})

In [8]:
hg1

Graph(num_nodes={'game': 1, 'user': 2},
      num_edges={('user', 'plays', 'game'): 2},
      metagraph=[('user', 'game', 'plays')])

In [9]:
hg2

Graph(num_nodes={'game': 3, 'user': 1},
      num_edges={('user', 'plays', 'game'): 3},
      metagraph=[('user', 'game', 'plays')])

In [10]:
hg3

Graph(num_nodes=2, num_edges=1,
      ndata_schemes={}
      edata_schemes={})

In [11]:
bhg12 = dgl.batch([hg1, hg2])
bhg12

Graph(num_nodes={'game': 4, 'user': 3},
      num_edges={('user', 'plays', 'game'): 5},
      metagraph=[('user', 'game', 'plays')])

In [12]:
# can't batch different schema graph
bhg13 = dgl.batch([hg1, hg3])
bhg13

DGLError: ignored

# how to batch graphs with different schema, a workaround

In [47]:
hg1 = dgl.heterograph({
    ('user', 'plays', 'game') : (th.tensor([0, 1]), th.tensor([0, 0])),
    ('user', 'follows', 'user') : [],
    ('company', 'own', 'game') : []
    })
hg1.nodes['user'].data['feature']=torch.randn(2, 10)
hg1.nodes['game'].data['feature']=torch.randn(1, 8)



hg3 = dgl.heterograph({
    ('user', 'follows', 'user') : (th.tensor([0]), th.tensor([1])),
    ('user', 'plays', 'game') : [],
    ('company', 'own', 'game') : []
    })
hg3.nodes['user'].data['feature']=torch.randn(2, 10)

In [48]:
hg1

Graph(num_nodes={'company': 0, 'game': 1, 'user': 2},
      num_edges={('company', 'own', 'game'): 0, ('user', 'follows', 'user'): 0, ('user', 'plays', 'game'): 2},
      metagraph=[('company', 'game', 'own'), ('user', 'user', 'follows'), ('user', 'game', 'plays')])

In [49]:
hg1.ndata

defaultdict(<class 'dict'>, {'feature': {'game': tensor([[-0.0165,  1.2051, -0.8798, -0.4483, -0.2962,  1.2297, -0.1893,  0.5736]]), 'user': tensor([[-0.3241, -1.1517, -0.1897,  1.4397,  0.0798, -0.2200, -0.7886,  0.3273,
         -0.5622,  0.3774],
        [-0.5301,  0.3028,  0.1678, -1.1464,  0.3405,  0.8191, -0.2724, -0.0692,
         -1.1075, -0.2335]])}})

In [50]:
hg3

Graph(num_nodes={'company': 0, 'game': 0, 'user': 2},
      num_edges={('company', 'own', 'game'): 0, ('user', 'follows', 'user'): 1, ('user', 'plays', 'game'): 0},
      metagraph=[('company', 'game', 'own'), ('user', 'user', 'follows'), ('user', 'game', 'plays')])

In [51]:
bhg13 = dgl.batch([hg1, hg3])
bhg13

Graph(num_nodes={'company': 0, 'game': 1, 'user': 4},
      num_edges={('company', 'own', 'game'): 0, ('user', 'follows', 'user'): 1, ('user', 'plays', 'game'): 2},
      metagraph=[('company', 'game', 'own'), ('user', 'user', 'follows'), ('user', 'game', 'plays')])